# Loom construction

This notebook takes in the `kb` outputs in `mtx` format for Allen and 10x datasets, converts the gene IDs to gene names, and exports spliced, unspliced, and ambiguous counts to the `loom` format.

In [3]:
import loompy as lp
import scipy
import numpy as np
import matplotlib.pyplot as plt

In [27]:
import pandas as pd

In [10]:
mtx_names = ['cells_x_genes{}.mtx'.format(x) for x in ('','.2','.ambiguous')]
data_names = ('spliced','unspliced','ambiguous')
bc_name = 'cells_x_genes.barcodes.txt'
gene_name = 'cells_x_genes.genes.txt'

In [57]:
gene_ref_loc = '/home/ggorin/ref/refdata-gex-mm10-2020-A/kallisto-D/ref.t2g'
gene_ref = pd.read_csv(gene_ref_loc,header=None,sep='\t')
gene_ref = gene_ref.iloc[:,1:3]
gene_ref.columns = ['gene_id','gene_name']
gene_ref.set_index('gene_id',inplace=True)

/home/ggorin/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [58]:
gene_ref

,gene_name
gene_id,
ENSMUSG00000051951.5,Xkr4
ENSMUSG00000051951.5,Xkr4
ENSMUSG00000051951.5,Xkr4
ENSMUSG00000089699.1,Gm1992
ENSMUSG00000102331.1,Gm19938
...,...
ENSMUSG00000079808.3,AC168977.1
ENSMUSG00000095041.7,AC149090.1
ENSMUSG00000063897.3,CAAA01118383.1


In [62]:
gn = gene_ref.loc['ENSMUSG00000051951.5']['gene_name'].unique()

In [64]:
gn[0]

'Xkr4'

In [51]:

gene_ref['gene_id'].isin(genes).sum()

150438

In [8]:
spliced_layer = 'spliced'
unspliced_layer = 'unspliced'
ambiguous_layer = 'ambiguous'
gene_attr = 'gene_name'
cell_attr = 'barcode'

attribute_names=[(unspliced_layer,spliced_layer,ambiguous_layer),gene_attr,cell_attr]


In [14]:
indir = '/home/ggorin/dlist_count_data/'
outdir = '/home/ggorin/dlist_count_data/loom_kb/'

In [80]:
dataset_names = ['allen_'+x for x in ('A02','C01','B08','F08','H12')] \
               +['brain_5k_v3','brain_nuc_5k_v3']
dataset_dirs = [indir + x + '/counts_unfiltered/' for x in dataset_names]

In [19]:
#                 lp.create(outdir+'/allen_{}_{}.loom'.format(d,celltype_abbr[celltype]),\
#                           layers={'':scipy.sparse.csr_matrix(S_.shape),
#                                   spliced_layer:S_,
#                                   unspliced_layer:U_},\
#                                   row_attrs={gene_attr:gene_names},\
#                                   col_attrs={cell_attr:bc_filt})


In [73]:
for dataset in dataset_dirs[:1]:
#     bcs = np.loadtxt(dataset+'/'+bc_name,dtype=str)
    gene_ids = np.loadtxt(dataset+'/'+gene_name,dtype=str)

In [74]:
genes = ['']*len(genes)
for k,gene in enumerate(gene_ids):
    gn = gene_ref.loc[gene]['gene_name'].unique()
    if len(gn)==1:
        genes[k] = gn[0]
    else:
        genes[k] = 'nonunique'

In [87]:
# dataset

In [88]:
for k,dataset in enumerate(dataset_dirs):
    bcs = np.loadtxt(dataset+'/'+bc_name,dtype=str)
    gene_ids_ = np.loadtxt(dataset+'/'+gene_name,dtype=str)
    if ~np.all(gene_ids_==gene_ids):
        raise ValueError('IDs wrong!')
    l1 = scipy.io.mmread(dataset+mtx_names[0]).T
    l2 = scipy.io.mmread(dataset+mtx_names[1]).T
    l3 = scipy.io.mmread(dataset+mtx_names[2]).T
    lp.create(outdir + dataset_names[k]+'.loom',\
          layers={'':scipy.sparse.csr_matrix(l1.shape),
                  data_names[0]:l1,
                  data_names[1]:l2,
                  data_names[2]:l3},\
                  row_attrs={gene_attr:genes},\
                  col_attrs={cell_attr:bcs})


In [86]:
# gene_ids